In [1]:
from core.utils.configuration import Config

with Config:
    # settings for dataset
    Config.Dataset.ImagesRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\images'
    Config.Dataset.MasksRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\masks'
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    #train dataset
    Config.Dataset.Train.BatchSize = 8
    Config.Dataset.Train.Transform.Combination.Version = 'v1'
    Config.Dataset.Train.Transform.Combination.Components = 'default'
    Config.Dataset.Train.Transform.Combination.Params = 'default'
    Config.Dataset.Train.Transform.Combination.Schedules = 'default'
    Config.Dataset.Train.Transform.Schedule = 0.8
    #validation dataset
    Config.Dataset.Validation.BatchSize = 8
    Config.Dataset.Validation.Ratio = 0.05
    Config.Dataset.Validation.Transform.Combination.Version = 'v1'
    Config.Dataset.Validation.Transform.Combination.Components = 'default'
    Config.Dataset.Validation.Transform.Combination.Params = 'default'
    Config.Dataset.Validation.Transform.Combination.Schedules = 'default'
    Config.Dataset.Validation.Transform.Schedule = 0.8
    
    #choose training structure, including the model, loss, metrics, optimizer, schedular
    Config.Model.Task = 'SimpleSeg'
    # model
    Config.Model.Backbone.Name = 'Unet'
    Config.Model.Backbone.Param = dict(encoder_name='resnet50', encoder_weights='imagenet', in_channels=1, out_channels=4)
    Config.Model.Head.Name = 'Head_1'
    Config.Model.Head.Param = dict(logit_output=True, in_channels=4)
    
    # loss
    Config.Training.Losses.Name = ['DiceLoss','BCELoss']
    Config.Training.Losses.Param = [{},{}]
    Config.Training.Losses.Schedule = [1,0.2]
    # optimizer
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = dict(lr=0.001)
    # scheduler
    Config.Training.Scheduler.Name = 'Custom_1'
    Config.Training.Scheduler.Param = dict(warmup_epochs=1,reduce_gamma=-2)
    # metrics
    Config.Training.Metrics.Name = ['DiceLoss','BCELoss','DiceAccuracy']
    Config.Training.Metrics.Param = [dict(use_logit=True,w_bce=0.2),dict(use_logit=True,threshold=0.5)]
    
    # whether to freeze backbone
    Config.Training.Settings.Model.FreezeBackbone = False
    
    # setup training process
    Config.Training.Settings.Epochs = 30
    Config.Training.Settings.GradientAccumulation = 1
    Config.Training.Settings.AmpScaleTrain = True
    # set random property
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Settings.Random.Seed.Dataset.Split = 94
    Config.Training.Settings.Random.Seed.Dataset.Transform = 180
    Config.Training.Settings.Random.Seed.Dataset.Shuffle = 98
    Config.Training.Settings.Random.Seed.Model = 53
    
    Config.Logging.StepsPerLog = 8
    Config.Logging.Image.Columns = 3
    Config.Logging.Image.Rows = 10
    Config.Logging.Image.Figsize = (300,300)
    Config.Logging.Image.Fontsize = 200
    Config.Logging.Image.DPI = 10
    Config.Logging.Image.MaskAlpha = 0.4
    Config.Logging.Image.Threshold = 0.5
    
    Config.Logging.RootPath = 'Results'
    Config.Logging.FolderName = 'resnet_pretrained'
    Config.Logging.TrialNumber = 3
    Config.Logging.Note = ''

In [1]:
import core.utils.configuration

Config = core.utils.configuration.initialize_config()

with Config:
    Config.Data.Type = 'paired_images_with_contrast_label'
    Config.Data.Image.Preprocess = 'GrayImage' #'GrayImage' 'RGBImage' 'BGRImage' 
    Config.Data.Image.Postprocess = 'CLAHE_float' #'CLAHE_float' 'CLAHE_int'
    
    Config.Model.Task = 'SimpleSeg'
    Config.Model.Param = dict(
        backbone_name = 'UniversalUnet', #'UniversalUnet' 'EffiNetB4Unet_AttDs' 'EffiNetB4Unet_AttRes'
        backbone_param = dict(
            encoder_name='resnet50',
            encoder_weights='imagenet',
            in_channels=1,
            out_channels=4),
        seg_head_name = 'Head_1',
        seg_head_param = dict(in_channels=4)
        )
    
    Config.Training.Settings.TrailNumber = 1
    Config.Training.Settings.Epochs = 30
    Config.Training.Settings.AmpScaleTrain = True
    
    Config.Training.Settings.Random.RootSeed = 100
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    
    Config.Training.Dataset.Path = r'D:\Datasets\Cardiac_Catheterization\train'
    Config.Training.Dataset.NumWorkers = 0
    Config.Training.Dataset.PinMemory = False
    Config.Training.Dataset.PrefetchFactor = 2
    Config.Training.Dataset.Train.BatchSize = 8
    Config.Training.Dataset.Val.Ratio = 0.1
    Config.Training.Dataset.Val.BatchSize = 16
    Config.Training.Dataset.Augmentation.Prob = 'Constant(0.8)'
    Config.Training.Dataset.Augmentation.Content = [
        'GridDistortion             -p:Constant(0.8)',
        'ElasticTransform           -p:Constant(0.8)',
        'Affine                     -p:Constant(0.8)',
        'GaussNoise                 -p:Constant(0.8)',
        'Blur                       -p:Constant(0.8)',
        'Downscale                  -p:Constant(0.8)',
        'RandomBrightnessContrast   -p:Constant(0.8)'
    ]
    
    Config.Training.Losses = [
        'BCELoss2D  -subject:"mask"           -weights:"contrast_exist" -schedule:WarmUp(warmup_epochs=5,max_value=2)  -smooth:1',
        'DiceLoss   -subject:"mask"           -weights:"contrast_exist" -schedule:Constant(1)',
        'BCELoss    -subject:"contrast_exist" -weights:None             -schedule:Constant(1)'
    ]
    
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = {'lr':0.001}
    Config.Training.LearningRateSchedule.Name = ''
    Config.Training.LearningRateSchedule.Param = {}
    
    Config.Training.Metrics = [
        'BinaryAccuracy2D   -subject:"mask"           -weights:"contrast_exist" -threshold:0.5  -smooth:1',
        'DiceAccuracy       -subject:"mask"           -weights:"contrast_exist" -threshold:0.5',
        'BinaryAccuracy     -subject:"contrast_exist" -weights:None             -threshold:0.5'
    ]
    
    Config.Record.Frequence = 8
    Config.Record.RootPath = 'Results'
    Config.Record.FolderName = 'resnet_pretrained'
    Config.Record.Note = ''

In [28]:
import torch

model = torch.nn.Sequential(torch.nn.Conv2d(5,10,3))

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [34]:
model = model.to(device)

In [40]:
x = torch.ones((2,5,5,5),dtype=float)
x = x.to(device)
x

tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1.,

In [6]:
import pathlib

list(float(x.with_suffix('').name.replace('pred_','')) for x in pathlib.Path(r'D:\Datasets\Cardiac_Catheterization\新增資料夾').glob('pred_*.png'))

[0.85, 0.86]

In [7]:
f'{None:.4f}'

TypeError: unsupported format string passed to NoneType.__format__

In [12]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('xxx')
r = 5
for i in range(100):
    writer.add_scalar('run_15h',1,i)
    writer.add_scalars('run_15h', {'xsinx':i*np.sin(i/r),
                                    'xcosx':i*np.cos(i/r),
                                    'tanx': np.tan(i/r)}, i)
writer.close()